In [2]:
import pandas as pd
import numpy as np

## Worklist prioritization: Emergency Setting

In [3]:
## First, read in the file of the current worklist with the probabilities that your two algorithms have
## generated for the two types of findings you're most concerned with:

worklist = pd.read_csv('probabilities.csv')

In [4]:
worklist.head()

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability
0,chest_xray,0.0,0.05
1,chest_xray,0.0,0.17
2,chest_xray,0.0,0.00
3,chest_xray,0.0,0.04
4,wrist_xray,0.0,0.00


In [8]:

# Step 1: Define the worklist data
data = {
    "Image_Type": ["chest xray", "chest xray", "chest xray", "chest xray", "wrist xray"],
    "Brain_bleed_probability": [0.0, 0.0, 0.0, 0.0, 0.0],
    "Aortic_dissection_probability": [0.05, 0.17, 0.0, 0.04, 0.0]
}
worklist = pd.DataFrame(data)
worklist

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability
0,chest xray,0.0,0.05
1,chest xray,0.0,0.17
2,chest xray,0.0,0.00
3,chest xray,0.0,0.04
4,wrist xray,0.0,0.00


In [10]:
# Step 2: Compute original read times (6 min per image, FIFO order)
worklist["Original_Index"] = worklist.index
worklist["Original_Read_Time"] = worklist["Original_Index"] * 6
worklist

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability,Original_Index,Original_Read_Time
0,chest xray,0.0,0.05,0,0
1,chest xray,0.0,0.17,1,6
2,chest xray,0.0,0.00,2,12
3,chest xray,0.0,0.04,3,18
4,wrist xray,0.0,0.00,4,24


In [12]:
# Step 3: Compute combined priority score (brain_bleed + aortic_dissection)
worklist["Priority_Score"] = (
    worklist["Brain_bleed_probability"] + worklist["Aortic_dissection_probability"]
)
worklist

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability,Original_Index,Original_Read_Time,Priority_Score
0,chest xray,0.0,0.05,0,0,0.05
1,chest xray,0.0,0.17,1,6,0.17
2,chest xray,0.0,0.00,2,12,0.00
3,chest xray,0.0,0.04,3,18,0.04
4,wrist xray,0.0,0.00,4,24,0.00


In [14]:
# Step 4: Re-order based on descending priority score (stable sort)
worklist_sorted = worklist.sort_values(
    by=["Priority_Score", "Original_Index"], ascending=[False, True]
).reset_index(drop=True)
worklist_sorted

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability,Original_Index,Original_Read_Time,Priority_Score
0,chest xray,0.0,0.17,1,6,0.17
1,chest xray,0.0,0.05,0,0,0.05
2,chest xray,0.0,0.04,3,18,0.04
3,chest xray,0.0,0.00,2,12,0.00
4,wrist xray,0.0,0.00,4,24,0.00


In [16]:
# Step 5: Assign new read times based on new order
worklist_sorted["New_Read_Time"] = worklist_sorted.index * 6
worklist_sorted

,Image_Type,Brain_bleed_probability,Aortic_dissection_probability,Original_Index,Original_Read_Time,Priority_Score,New_Read_Time
0,chest xray,0.0,0.17,1,6,0.17,0
1,chest xray,0.0,0.05,0,0,0.05,6
2,chest xray,0.0,0.04,3,18,0.04,12
3,chest xray,0.0,0.00,2,12,0.00,18
4,wrist xray,0.0,0.00,4,24,0.00,24


In [17]:
# Step 6: Merge reordered read times back to original dataframe
merged = worklist.merge(
    worklist_sorted[["Original_Index", "New_Read_Time"]],
    on="Original_Index",
    how="left"
)

In [18]:
# Step 7: Calculate time delta
merged["Time_Delta_Minutes"] = merged["New_Read_Time"] - merged["Original_Read_Time"]


In [19]:
# Step 8: Final output
display_cols = [
    "Original_Index", "Image_Type", "Brain_bleed_probability", "Aortic_dissection_probability",
    "Priority_Score", "Original_Read_Time", "New_Read_Time", "Time_Delta_Minutes"
]
merged[display_cols]

,Original_Index,Image_Type,Brain_bleed_probability,Aortic_dissection_probability,Priority_Score,Original_Read_Time,New_Read_Time,Time_Delta_Minutes
0,0,chest xray,0.0,0.05,0.05,0,6,6
1,1,chest xray,0.0,0.17,0.17,6,0,-6
2,2,chest xray,0.0,0.00,0.00,12,18,6
3,3,chest xray,0.0,0.04,0.04,18,12,-6
4,4,wrist xray,0.0,0.00,0.00,24,24,0
